In [ ]:
#@title Your Info { display-mode: "form" }

Name = '' #@param {type:"string"}

# DataRes Research Coding Assessment

## Due by 11:59pm, April 5

Thanks for taking the time to apply to the research team at DataRes! This assessment consists of handful of machine learning exercises that use PyTorch. If you aren't familiar with this library, there's lots of helpful documentation [here](https://pytorch.org/docs/stable/index.html), and **I highly recommend reading it thoroughly if you get stuck** to make sure you're passing correct types/dimensions into the arguments. Lots of code has already been written for you, just make sure to complete sections marked "TODO:" or "Your Implementations". There are a number of challenging problems to work on, so if you have any questions, please feel free to reach out to [Larry](mailto:larrylu0302@g.ucla.edu) at any time! Lastly, please feel free to submit the assessment even if you can't finish it all, I will take the time to look at every application I recieve.

Make sure to submit this completed assessment by 11:59pm, April 5. Submissions will be turned in by emailing this notebook as an .ipynb and the .json with your model's results (more on this below) to me.

Good luck!

### Before you begin

Here are some things to keep in mind:

* Everything should be done using the PyTorch library
* Reading documentation can be very helpful if you're stuck
* If you're really struggling on a section, send me an email and move on to another one in the meantime
* Currently, the trainings are initialized to default epochs. Colab is about half as fast as Kaggle, so depending on which one you're using, you can change the number of epochs so training doesn't take forever (~10 minutes is good)
* Sometimes you may get an error about the GPU/CUDA. If this happens, try changing the variables by adding this line: x = x.to('cuda') and restarting

### Overview

In this assessment, you will be working with the [MiniPlaces dataset](https://github.com/CSAILVision/miniplaces), a dataset of scene images (10+ million images) with a wide variety of real-world environments (400+ unique scene categories). The MiniPlaces dataset is a subset of the [Places2 dataset](http://places2.csail.mit.edu/) and contains 100,000 images for training, 10,000 images for validation, and 10,000 images for testing, each of which has been annotated with one of 100 different scene categories. These images are divided into three folders: train, val, and test.

Question 1 will use only some categories of this dataset. We've called this new dataset TinyPlaces. Questions 2-4 will use MiniPlaces (all subcategories).

You will be completing machine learning exercises in which you will train models using this dataset. These exercises touch on the following topics:


*   Linear, Logistic, and Softmax Regression
*   Multi-Layer Perceptrons
*   Convolutional Neural Networks (CNNs)

Don't worry about having to deal with preprocessing the data... we've already written code to do that for you! Just run the cell below.



In [ ]:
import os
path = './'
os.makedirs(os.path.join(path, 'DataRes_Research_Assessment', 'data'), exist_ok=True)
root_dir = os.path.join(path, 'DataRes_Research_Assessment')


!pip3 install --upgrade gdown --quiet
!gdown 1z3B1GR7UtHZGrqNjUaep6thHwrN3IYSI

import tarfile
from tqdm import tqdm

tar = tarfile.open("data.tar.gz", "r:gz")
total_size = sum(f.size for f in tar.getmembers())
with tqdm(total=total_size, unit="B", unit_scale=True, desc="Extracting tar.gz file") as pbar:
    for member in tar.getmembers():
        tar.extract(member, os.path.join(root_dir, 'data'))
        pbar.update(member.size)
tar.close()

import urllib.request

import os
import pickle

import numpy as np

def load_subcategories(txt_path):
    subcategories = {}
    file = open(txt_path, 'r')
    lines = file.readlines()
    for i, l in enumerate(lines):
        info = l.split()
        info[0] = info[0][3:]
        subcategories.update({info[0]: {'ori_class_id': int(info[1]), 'class_id': i}})

    return subcategories


from tqdm import trange
import cv2
import random

def select_samples(subcategories, root_dir, split, n_images_per_subcategory):
    samples = []
    if split == "train":
        train_dir = os.path.join(root_dir, "data", "images", "train")
        for i in subcategories:
            child_dir = os.path.join(train_dir, i[0], i)
            pics = random.sample(os.listdir(child_dir), n_images_per_subcategory)
            for j in pics:
                samples.append((cv2.resize(cv2.imread(os.path.join(child_dir, j)), (32,32)).flatten().tolist(), subcategories[i]["class_id"]))
    elif split == "val":
        val_dir = os.path.join(root_dir, "data", "images")
        file = open(os.path.join(root_dir, "data", "val.txt"), 'r')
        lines = file.readlines()
        val_data = []
        for i in lines:
            val_data.append(i.split())
        random.shuffle(val_data)
        for i in subcategories:
            old_id = subcategories[i]["ori_class_id"]
            count = 0
            for j in val_data:
                if int(j[1]) == old_id:
                    samples.append((cv2.resize(cv2.imread(os.path.join(val_dir, j[0])), (32,32)).flatten().tolist(), subcategories[i]["class_id"]))
                    count += 1
                if count >= n_images_per_subcategory:
                    break
    return samples

def create_tinyplaces(samples, binary=True):
    data, labels = [], []
    for i in samples:
        data.append(i[0])
        if binary:
            if i[1] >= 10:
                labels.append(1)
            else:
                labels.append(0)
        else:
            labels.append(i[1])
    data = np.array(data)
    labels = np.array(labels)
    dataset = {"data": data, "label": labels}

    return dataset


# Set the root directory of the dataset
root_dir = './DataRes_Research_Assessment/data'

# Load the target subcategories and their class IDs
subcategories = load_subcategories(os.path.join(root_dir, 'data', 'categories_tinyplaces.txt'))

# Select the samples from the train split of the TinyPlaces dataset
train_samples = select_samples(subcategories, root_dir, 'train', 500)

# Create the TinyPlaces datasets for binary and multiclass classification
tinyplaces_binary_train = create_tinyplaces(train_samples, binary=True)
tinyplaces_multi_train = create_tinyplaces(train_samples, binary=False)

# Select the samples from the val split of the MiniPlaces dataset
val_samples = select_samples(subcategories, root_dir, 'val', 50)

# Create the TinyPlaces datasets for binary and multiclass classification
tinyplaces_binary_val = create_tinyplaces(val_samples, binary=True)
tinyplaces_multi_val = create_tinyplaces(val_samples, binary=False)

# Save the TinyPlaces datasets to the data directory
data_dir = os.path.join(root_dir, 'data')


with open(os.path.join(data_dir, 'tinyplaces_binary_train.pkl'), 'wb') as f:
    pickle.dump(tinyplaces_binary_train, f)

with open(os.path.join(data_dir, 'tinyplaces_multi_train.pkl'), 'wb') as f:
    pickle.dump(tinyplaces_multi_train, f)

with open(os.path.join(data_dir, 'tinyplaces_binary_val.pkl'), 'wb') as f:
    pickle.dump(tinyplaces_binary_val, f)

with open(os.path.join(data_dir, 'tinyplaces_multi_val.pkl'), 'wb') as f:
    pickle.dump(tinyplaces_multi_val, f)


class TinyPlacesDataset(object):
    def __init__(self, data_dict):
        self.dataset = data_dict
        self.num_samples = len(data_dict['data'])

    def subsample(self, ratio=0.1, seed=None):
        if seed is not None:
            np.random.seed(seed)

        nums = random.sample(range(self.num_samples), int(ratio * self.num_samples))
        sub_dataset = {'data': self.dataset['data'][nums], 'label': self.dataset['label'][nums]}
        subsampled_dataset = TinyPlacesDataset(sub_dataset)

        return subsampled_dataset

## GPU

You will train your models using the GPU, which enables parallel processing and is vital for training deep learning models.

If you're using Colab, go to:
Runtime -> Change Runtime Type -> Hardware Accelerator -> GPU

If you're using Kaggle, go to: More Settings (the three dots) -> Accelerator -> GPU P100

In [ ]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if device == torch.device('cuda'):
    print(f'Using device: {device}. Good to go!')
else:
    print('Please set GPU via Edit -> Notebook Settings.')

In [ ]:
# This line of code gives you info about GPU
! nvidia-smi

In [ ]:
with open(os.path.join(root_dir, 'data', 'tinyplaces_binary_train.pkl'), 'rb') as f:
    binary_train = TinyPlacesDataset(pickle.load(f))
with open(os.path.join(root_dir, 'data', 'tinyplaces_binary_val.pkl'), 'rb') as f:
    binary_val = TinyPlacesDataset(pickle.load(f))
with open(os.path.join(root_dir, 'data', 'tinyplaces_multi_train.pkl'), 'rb') as f:
    multi_train = TinyPlacesDataset(pickle.load(f))
with open(os.path.join(root_dir, 'data', 'tinyplaces_multi_val.pkl'), 'rb') as f:
    multi_val = TinyPlacesDataset(pickle.load(f))

# Convert everything from numpy arrays to tensors and move them to the GPU using .cuda()
for dataset in [binary_train, binary_val, multi_train, multi_val]:
    for k in ['data', 'label']:
        dataset.dataset[k] = torch.tensor(dataset.dataset[k]).float().cuda()

In [ ]:
from tqdm import tqdm

## Q1 Regressions

### Linear Regression

Below, you will implement linear regression. You'll need to use PyTorch to initialize the parameters and implement the linear function and mean squared error.

If the prediction score > 0.5, we consider the image to be of outdoor category. Otherwise, we consider it to be indoor category.

In [ ]:
class LinearRegression(object):
    def __init__(self, input_size, output_size):
        """
          Initialize the weights and biases using zeros distribution

          Parameters:
              input_size (int): The input size (dimension of feature vectors)
              output_size (int): The output size (dimension of output logits)

          Returns:
              None.
        """
        # Initialize the weights and biases using zeros
        # Make sure your tensors keep track of their gradient
        # Move the parameters to GPU (cuda)
        ################# Your Implementations #################################



        ################# End of your Implementations ##########################

    def linear(self, x):
        ################# Your Implementations #################################



        ################# End of your Implementations ##########################
        return output

    def forward(self, x):
        # To make the output shape compact.
        return self.linear(x).squeeze()

    def get_loss(self, pred_logits, targets):
        # Calculate the mean squared error between the predicted labels and the ground-truth labels
        loss = None
        ################# Your Implementations #################################



        ################# End of your Implementations ##########################
        return loss

    def fit(self, x, y, x_val, y_val, lr, epochs=500000, print_freq=1000):
        # Fit the linear regression model to the training data using gradient descent
        # x is the input data, y is the ground-truth labels
        # lr is the learning rate, epochs is the number of epochs

        # To store validation accuracy
        val_accs = []
        # Create a progress bar using tqdm
        pbar = tqdm(range(epochs))
        for epoch in pbar:
            # Calculate the loss
            y_pred_logits = self.forward(x)
            loss = self.get_loss(y_pred_logits, y)
            # Backpropagate the loss to compute the gradients
            loss.backward()
            # Update the weights and biases using gradient descent
            with torch.no_grad():
                self.W -= lr * self.W.grad
                self.b -= lr * self.b.grad
                # Reset the gradients
                self.W.grad.zero_()
                self.b.grad.zero_()

            if epoch % print_freq == 0:
                # Calculate the validation accuracy
                val_acc = self.evaluate(x_val, y_val)
                val_accs.append(val_acc)
                # Update the progress bar with the validation accuracy and training loss
                pbar.set_description(f'val_acc: {val_acc:.3f}')
        return val_accs

    def evaluate(self, x, y):
        # Evaluate the performance of the linear regression model on the dataset
        # x is the input data, y is the ground-truth labels
        # Calculate the predicted labels
        y_pred = self.forward(x) > 0.5
        return (y_pred == y).float().mean().item()

In [ ]:
def normalize(x):
    # We can simply divide x by 255 since its range is (0,255)
    return x / 255.

In [ ]:
# Here "ori" indicates "original"
X_train_ori, y_train = binary_train.dataset['data'], binary_train.dataset['label']
X_val_ori, y_val = binary_val.dataset['data'], binary_val.dataset['label']

# Normalization
X_train = normalize(X_train_ori)
X_val = normalize(X_val_ori)

Use the next 3 cells to debug. The next cell should give an accuracy of 0.5 since we're using zeros-initialization.

In [ ]:
linear_model = LinearRegression(3072, 1)
train_acc = linear_model.evaluate(X_train.double(), y_train)
val_acc = linear_model.evaluate(X_val.double(), y_val)
print('train accuracy:', train_acc)
print('val accuracy:', val_acc)

In [ ]:
# Try to debug using this cell
linear_model = LinearRegression(3072, 1)
# We refer the raw outputs from a model to as "logits",
# i.e., we haven't transformed the results to binary labels.
y_pred_logits = linear_model.forward(X_train.double())
loss = linear_model.get_loss(y_pred_logits, y_train)
print("loss:", loss.item())

In [ ]:
# compute gradients
loss.backward()

# check the gradients
print(linear_model.W.grad)
print(linear_model.b.grad)

Great, you've fully implemented the linear regression model! Let's train it.

In [ ]:
linear_model = LinearRegression(3072, 1)
# It may be helpful to change the number of epochs to a very small number while debugging
# For your final submission, you can change the epochs so that training doesn't take forever (~10 minutes is good)
lin_val_accs = linear_model.fit(X_train.double().cuda(), y_train, X_val.double().cuda(), y_val, 1e-4, 500000, 1000)

### Logistic Regression

Logistic regression is very similar to linear regression, but the sigmoid function is added to the forward pass and cross-entropy is used for loss. Here's the formulas:

Sigmoid:

$$\frac{1}{1+e^{-x}}$$

Cross-Entropy Loss:

$$\mathrm{CE_Loss}(p, y) = -{(y\log(p) + (1-y)\log(1-p))}$$

Implement them below!

In [ ]:
def sigmoid(x):
    ################# Your Implementations #################################



    ################# End of your Implementations ##########################
    return output


def cross_entropy_loss(p, y):
    ################# Your Implementations #################################



    ################# End of your Implementations ##########################
    return output

Use these implementations below to implement the Logistic Regression model.

In [ ]:
class LogisticRegression(LinearRegression):
    def __init__(self, input_size, output_size):
        super(LogisticRegression, self).__init__(input_size, output_size)

    def forward(self, x):
        # Apply the sigmoid function to the linear output
        ################# Your Implementations #################################



        ################# End of your Implementations ##########################
        return output

    def get_loss(self, pred_logits, targets):
        # Calculate the cross-entropy loss
        ################# Your Implementations #################################



        ################# End of your Implementations ##########################
        return loss

Train!

In [ ]:
logistic_model = LogisticRegression(3072, 1)
# Again, you can change the epochs so that training doesn't take forever (~10 minutes is good)
logi_val_accs = logistic_model.fit(X_train.double(), y_train, X_val.double(), y_val, 1e-4, 500000, 1000)

Plot the validation accuracies of both models. Logistic Regression should be more accurate.

In [ ]:
import matplotlib.pyplot as plt

plt.plot(lin_val_accs, label='LinearRegression')
plt.plot(logi_val_accs, label='LogisticRegression')
plt.legend()
plt.xlabel('Iteration')
plt.ylabel('Validation Accuracy (%)')
plt.show()

### Softmax Regression

Softmax regression is similar to logistic regression except it uses the softmax function instead of sigmoid and uses negative log likelihood for the loss. Here's the formulas:

Softmax:

$$\frac{e^{z_{i}}}{\sum_{j=1}^K e^{z_{j}}} \ \ \ for\ i=1,2,\dots,K$$

where z is the output of the linear model, and K is the number of classes.

Negative Log Likelihood:

$$-{\log(p(y))}$$

where p(y) is the predicted probability of the data point belonging to the true class y.

Implement them below!

In [ ]:
def softmax(x):
    # Implement the softmax function
    # x is the input data with shape (batch_size, input_size)
    # Return the softmax output with shape (batch_size, output_size)
    ################# Your Implementations #################################



    ################# End of your Implementations ##########################
    return output

def nll_loss(pred_probs, targets):
    # Calculate the negative log likelihood loss
    # pred_probs is the predicted probability distribution with shape (batch_size, output_size)
    # targets is the ground-truth labels with shape (batch_size,)
    # Return the negative log likelihood loss with shape (batch_size,)

    # There are actually two functions for negative log likelihood loss: torch.nn.NLLLoss and torch.nn.functional.nll_loss
    # You can convert the ground-truth labels to one-hot encoding using torch.eye() and pass to NLLLoss
    # Alternatively, try converting targets to type LongTensor and passing that to functional.nll_loss
    ################# Your Implementations #################################



    ################# End of your Implementations ##########################
    return loss

Use these implementations below to implement the Softmax Regression model.

In [ ]:
class SoftmaxRegression(LinearRegression):
    def __init__(self, input_size, output_size):
        super(SoftmaxRegression, self).__init__(input_size, output_size)

    def forward(self, x):
        # Apply the softmax function to the linear output
        ################# Your Implementations #################################



        ################# End of your Implementations ##########################
        return output

    def get_loss(self, pred_logits, targets):
        # Calculate the cross-entropy loss
        ################# Your Implementations #################################



        ################# End of your Implementations ##########################
        return loss

    def evaluate(self, x, y):
        # Evaluate the performance of the linear regression model on the dataset
        # x is the input data, y is the ground-truth labels
        # Calculate the predicted labels
        y_pred = self.forward(x)
        y_pred = y_pred.argmax(dim=1) #changed from dim=0 to dim=1
        return (y_pred == y).float().mean().item()


In [ ]:
# Prepare multi-class dataset
# Here "ori" indicates "original"
X_train_ori_multi, y_train_multi = multi_train.dataset['data'], multi_train.dataset['label']
X_val_ori_multi, y_val_multi = multi_val.dataset['data'], multi_val.dataset['label']

# Normalization
X_train_multi = normalize(X_train_ori_multi)
X_val_multi = normalize(X_val_ori_multi)

Train! Don't worry if your accuracy is low, that's expected.

In [ ]:
sfm_model = SoftmaxRegression(3072, 20)
# You can change the epochs accordingly here as well
sfm_val_accs = sfm_model.fit(X_train_multi.double(), y_train_multi, X_val_multi.double(), y_val_multi, 1e-4, 100000, 1000)

### Comparison

In your own words, explain the differences between Linear Regression, Logistic Regression and Softmax Regression:

*TODO: Write your answer here*

## Q2: Multi-Layer Perceptron (MLP)

Now, we'll be using MiniPlaces, so there will be 100 different labels (instead of 20).

You will use PyTorch to implement linear layers and ReLu to create a multi-layer perceptron.

In [ ]:
from torch.utils.data import Dataset
from PIL import Image

class MiniPlaces(Dataset):
    def __init__(self, root_dir, split, transform=None, label_dict=None):
        assert split in ['train', 'val', 'test']
        self.root_dir = root_dir
        self.split = split
        self.transform = transform
        self.filenames = []
        self.labels = []
        self.label_dict = label_dict if label_dict is not None else {}
        if split == "train" or split == "val":
            with open(os.path.join(root_dir, ("train" if self.split == "train" else "val") + ".txt")) as f:
                for line in f:
                    line = line.rstrip().split()
                    n = int(line[0][-12:-4])
                    if n <= 900:
                        self.filenames.append(os.path.join(line[0]))
                        self.labels.append(int(line[1]))
        if label_dict is None and split == "train":
            with open(os.path.join(root_dir, "train.txt")) as f:
                num = -1
                for line in f:
                    line = line.rstrip().split()
                    if int(line[1]) > num:
                        num += 1
                        self.label_dict.update({int(line[1]): line[0][8:line[0].find("/", 8)]})
        if split == "test":
            self.labels = os.listdir(os.path.join(root_dir, "images", "test"))
            self.filenames = ["test/" + i for i in self.labels]

    def __len__(self):
        dataset_len = len(self.labels)
        return dataset_len

    def __getitem__(self, idx):
        image = Image.open(os.path.join(self.root_dir, "images", self.filenames[idx]))
        if not self.transform is None:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label

from torchvision import transforms

data_transform = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
])

data_root = os.path.join(root_dir, 'data')
miniplaces_train = MiniPlaces(data_root, split='train', transform=data_transform)
miniplaces_val = MiniPlaces(
    data_root, split='val',
    transform=data_transform,
    label_dict=miniplaces_train.label_dict)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class FastMLP(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        """
        Initialize an MLP classifier.
        Input and output sizes of each layer:
          1) fc1: input_size, hidden_size
          2) fc2: hidden_size, hidden_size
          3) fc3: hidden_size, num_classes

        Args:
            input_size (int): Size of the input layer.
            hidden_size (int): Size of the hidden layer.
            num_classes (int): Number of classes in the output layer.
        """
        super().__init__()
        self.fc1 = None
        self.fc2 = None
        self.fc3 = None
        ################# Your Implementations #################################
        # TODO: Define the layers of the MLP
        # Hint: The imported modules should give you a clue on which PyTorch functions to use



        ################# End of your Implementations ##########################

    def forward(self, x):
        """
        Forward pass of the MLP classifier.
        Using ReLU as the activation function after each layer, except for the output layer.
        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, input_size).

        Returns:
            torch.Tensor: Output tensor of shape (batch_size, num_classes).
        """
        ################# Your Implementations #################################
        # TODO: Implement the forward pass of the MLP classifier



        ################# End of your Implementations ##########################
        return x


Then, define the training and evaluation functions to train and test the MLP classifier (You don't need to modify this part):

In [ ]:
def train(model, train_loader, val_loader, optimizer, criterion, device, num_epochs):
    # Place model on device
    model = model.to(device)
    best_acc = 0
    flag = False
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        # Use tqdm to display a progress bar during training
        with tqdm(total=len(train_loader), desc=f'Epoch {epoch + 1}/{num_epochs}') as pbar:
            for inputs, labels in train_loader:
                # Move inputs and labels to device
                inputs = inputs.to(device)
                labels = labels.to(device)
                # Zero out gradients
                optimizer.zero_grad()
                # Compute the logits and loss
                logits = model(inputs)
                loss = criterion(logits, labels)
                # Backpropagate the loss
                loss.backward()
                # Update the weights
                optimizer.step()
                # Update the progress bar
                pbar.update(1)
                pbar.set_postfix(loss=loss.item())

        # Evaluate the model on the validation set
        avg_loss, accuracy = evaluate(model, val_loader, criterion, device)
        if best_acc > accuracy:
            if flag:
                print(f'Validation set: Average loss = {avg_loss:.4f}, Accuracy = {accuracy:.4f}')
                break
            else:
                flag = True
        else:
            best_acc = accuracy
            flag = False
        print(f'Validation set: Average loss = {avg_loss:.4f}, Accuracy = {accuracy:.4f}')

def evaluate(model, test_loader, criterion, device):
    """
    Evaluate the classifier on the test set.

    Args:
        model: classifier to evaluate.
        test_loader (torch.utils.data.DataLoader): Data loader for the test set.
        criterion (callable): Loss function to use for evaluation.
        device (torch.device): Device to use for evaluation.

    Returns:
        float: Average loss on the test set.
        float: Accuracy on the test set.
    """
    model.eval()  # Set model to evaluation mode

    with torch.no_grad():
        total_loss = 0.0
        num_correct = 0
        num_samples = 0

        for inputs, labels in test_loader:
            # Move inputs and labels to device
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Compute the logits and loss
            logits = model(inputs)
            loss = criterion(logits, labels)
            total_loss += loss.item()

            # Compute the accuracy
            _, predictions = torch.max(logits, dim=1)
            num_correct += (predictions == labels).sum().item()
            num_samples += len(inputs)

    # Compute the average loss and accuracy
    avg_loss = total_loss / len(test_loader)
    accuracy = num_correct / num_samples

    return avg_loss, accuracy

def predict(model, test_dataloader):
    """
    Evaluate the classifier on the test set.

    Args:
        model: classifier to evaluate.
        test_loader (torch.utils.data.DataLoader): Data loader for the test set.
        criterion (callable): Loss function to use for evaluation.
        device (torch.device): Device to use for evaluation.

    Returns:
        float: Average loss on the test set.
        float: Accuracy on the test set.
    """
    out = []
    for i in test_dataloader:
        pic = i[0]
        lab = torch.argmax(model.to('cpu')(pic))
        out.append(lab.item())

    return out

data_transform_flatten = transforms.Compose([data_transform, torch.flatten])

In [ ]:
model = FastMLP(
    input_size=3 * 64 * 64,
    hidden_size=1024,
    num_classes=len(miniplaces_train.label_dict))
optimizer = torch.optim.SGD(
    model.parameters(),
    lr=0.01,
    momentum=0.9)
criterion = nn.CrossEntropyLoss()

data_root = os.path.join(root_dir, 'data')
train_dataset = MiniPlaces(
    root_dir=data_root, split='train',
    transform=data_transform_flatten)
val_dataset = MiniPlaces(
    root_dir=data_root, split='val',
    transform=data_transform_flatten,
    label_dict=train_dataset.label_dict)

### !!! DON'T CHANGE NUM_WORKERS FROM 0 !!!
### Using the loader will crash the notebook

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=64, num_workers=0, shuffle=True)
val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=64, num_workers=0, shuffle=False)

Train!

In [ ]:
train(model, train_loader, val_loader, optimizer, criterion, device, num_epochs=2)

Don't worry if your accuracy is low, that's to be expected.

## Q3: Convolutional Neural Network (CNN)

This question is similar to the last, but you'll be implementing a convolutional neural network instead. You'll need to use convolutional layers, a max pooling layer, and dropout layers.

In [ ]:
class FastConv(nn.Module):
    def __init__(
        self,
        input_channels, conv_hidden_channels, conv_out_channels,
        input_size=(64,64),
        dropout_rate1=0.25, dropout_rate2=0.5,
        fc_out_channels=128, num_classes=100,
        kernel_size=3, stride=1, padding=1):
        """
        Args:
          input_channels (int): Number of channels in the input image.
          conv_hidden_channels (int): Number of channels in the first convolutional layer.
          conv_out_channels (int): Number of channels in the second convolutional layer.
          input_size (tuple, optional): Height and width of the input image. (default: (64,64))
          dropout_rate1, dropout_rate2 (float, optional): Dropout rate for
              the first/second dropout layer. (default: 0.25, 0.5)
          fc_out_channels (int, optional): Number of neurons in the first fully
              connected layer. (default: 128)
          num_classes (int, optional): Number of classes in the final output layer. (default: 100)
          kernel_size, stride, padding (int, optional): Parameters of convolutional layers.

        Initialize a convolutional neural network.
        You can use Pytorch's built-in nn.Conv2d function.
        Input and output shapes of each layer:
        1) conv1: (batch_size, input_channels, H, W) -> (batch_size, conv_hidden_channels, H, W)
        2) conv2: (batch_size, conv_hidden_channels, H, W) -> (batch_size, conv_out_channels, H, W)
        3) max_pooling: (batch_size, conv_out_channels, H, W)
        4) fc1: (batch_size, flatten_size) -> (batch_size, fc_out_channels)
        5) fc2: (batch_size, fc_out_channels) -> (batch_size, num_classes)
        """

        super().__init__()

        # I've given you 'flatten_size', since I'm not expecting you to have this formula memorized :)
        flatten_size = conv_out_channels * ((input_size[0] + 2 * padding - kernel_size) // stride + 1) * ((input_size[1] + 2 * padding - kernel_size) // stride + 1)
        self.conv1 = None
        self.conv2 = None
        self.max_pooling = None
        self.dropout1 = None
        self.dropout2 = None
        self.fc1 = None
        self.fc2 = None
        ################# Your Implementations #################################
        # TODO: Define the layers of the convolutional neural network
        # replace "None"s with your implementations.
        # All you need to do is to pass the input arguments to different constructors



        ################# End of your Implementations ##########################

    def forward(self, x, return_intermediate=False):
        """
        Forward pass of the convolutional neural network.
        The input tensor 'x' should pass through the following layers:
        1) conv1: (batch_size, input_channels, H, W) -> (batch_size, conv_hidden_channels, H, W)
        2) Apply relu.
        3) conv2: (batch_size, conv_hidden_channels, H, W) -> (batch_size, conv_out_channels, H, W)
        4) max_pooling: Perform max pooling on the output from conv2
        5) dropout1: Perform dropout on the output from max_pooling
        6) Flatten the output from dropout1
        7) fc1: Pass through a fully connected layer
        8) dropout2: Perform dropout on the output from fc1
        9) Apply relu.
        10) fc2: Pass the output from the actiction layer to through a fully connected
                layer to produce the final output
        """
        ################# Your Implementations #################################
        # TODO: Implement the forward pass of the convolutional neural network



        ################# End of your Implementations ##########################
        return x


In [ ]:
conv_train_dataset = MiniPlaces(
    root_dir=data_root, split='train',
    transform=data_transform)
conv_val_dataset = MiniPlaces(
    root_dir=data_root, split='val',
    transform=data_transform)
conv_train_loader = torch.utils.data.DataLoader(
    conv_train_dataset, batch_size=64, num_workers=0, shuffle=True)
conv_val_loader = torch.utils.data.DataLoader(
    conv_val_dataset, batch_size=64, num_workers=0, shuffle=False)

model = FastConv(
    input_channels=3, conv_hidden_channels=64, conv_out_channels=128,
    input_size=(64,64),
    dropout_rate1=0.25, dropout_rate2=0.5,
    fc_out_channels=128,
    kernel_size=3, stride=1, padding=1,
    num_classes=len(miniplaces_train.label_dict))
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = nn.CrossEntropyLoss()

Train!

In [ ]:
train(model, conv_train_loader, conv_val_loader, optimizer, criterion, device, num_epochs=2)

I can get ~20% accuracy after two epochs. How about you?

## Q4: Your own model

You will construct your own model using built-in convolutional layers. You can base your model on the provided `FastConv` class, or choose to modify the number of convolutional layers, feature size, learning rate, optimizer, and other parameters to suit your needs. You can also use any data transformations you'd like. You can even base your model off of an already existing architecture. The only restriction is that you can't blindly import an entire model at once; you should be creating every layer using PyTorch. Create your model and train using cells below.

In [ ]:
### Use these cells to design and train your model

In [ ]:
### Previous questions will be super useful

Now that you've designed your model and trained it, it's time to test it's accuracy! Use the "test" images and the 'predict' function.

NOTE: Make sure when constructing your DataLoader, you set 'shuffle' to 'False'. This is so I can match your model's predictions to test images to find how accurate its predictions are.

In [ ]:
### Create your DataLoader and use predict to get your model's predictions

Now that you have a list containing your predictions, you should create a file named "{your_name}.json", containing "{id: predicted_label}" pairs. A function to do this has been provided, just pass in the output of the predict function from the previous cell. You can find the file under the "data" folder by clicking on the folder icon in Colab, or in your working directory output in Kaggle

In [ ]:
import json
def preds_to_json(preds):
    json_dict = {}
    for i, n in enumerate(preds):
        json_dict[str('0000000'+str(i+1))[-8:]] = n
    with open(os.path.join(root_dir,'your_name.json'), 'w', encoding='utf-8') as f:
        #If you're using a Kaggle notebook, change root_dir to '/kaggle/working/'
        json.dump(json_dict, f, ensure_ascii=False, indent=4)

## 'test_preds' should be the output of 'predict' from the previous cell
preds_to_json(test_preds)

### Model Architecture Explanation

In the cell below, explain your model's architecture and how you came up with it. Feel free to include as many details as you want, including how you decided on a learning rate or data transformations.

*TODO: Write your answer here*

## You've Finished!!!

Great job completing the assessment! The root directory for this notebook should now include the .json with your test predictions. In order to submit this assessment, all you have to do is email this notebook as an .ipynb and the .json to larrylu0302@g.ucla.edu

Thanks for taking the time to work through these problems! I look forward to reading through your application. You should expect to hear back a few days after April 5th.